Assignment 2 - Extract the reviews from Dune 2021 using IMDB API
===

*Due: December 6 2022*

Assess which sentiment analysis method (AFINN, VADER, TEXTBLOB) and input (Raw, Preprocessed without stop words, Preprocessed with stop words) correlates better with the user rating from IMDB.

## Imports and functions

In [59]:
import jupyter_black
import os
import json
import pandas as pd
import contractions
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer


vader_sia = SentimentIntensityAnalyzer()
afinn = Afinn()


jupyter_black.load()


data_path = "data/01_data.json"

movie = "dune_2021"


def create_wordstrings(df):
    """
    returns a clean wordstring
    """

    word_strings = df.quotes.str.lower().map(lambda x: contractions.fix(x))

    return word_strings

## Load reviews from disk

Download in Assignment 1

In [60]:
if os.path.exists(data_path):

    with open(data_path) as fp:
        data = json.load(fp)

    if movie in data.keys():
        print("Reviews loaded.")
        df = pd.json_normalize(data[movie])
        df["quoteClean"] = create_wordstrings(df)

    else:
        print("Movie not downloaded. Go to assignment 1 to download it.")
else:
    print("No data file available. Go to assignment 1.")

Reviews loaded.


## Add sentiment scores

In [68]:
df["afinn"] = df.quoteClean.apply(lambda x: afinn.score(x))
df["vader"] = df.quoteClean.apply(lambda x: vader_sia.polarity_scores(x)["compound"])
# TEXTBLOB missing

<div class="alert alert-warning">Textblob scores still missing</div>

## Compute correlations with ratings

How do the ratings look like?

In [73]:
df.scoreOri.unique()

array(['4/5', '5/5', '9/10', 'B+', '8/10', nan, 'B-', '2/5', 'A', '3.5/5',
       'D', '4/4', '3/5', '7/10', '10/10', '3/4', '8.5/10', '3.5',
       '4.5/5', '3.5/4', '', '1.5/4', '2.5/5', 'A-', '2.5/4', '6.5/10',
       'C', 'B', 'C-', '2/4', '4', '6/10', 'C+', '5.5/10', 'A+',
       '2.5 out of 5', '7.5/10', '1/4', '4.5/10', '89/100', '7',
       '2.5/4.0', '9.5/10', '4.84458/10', '4/10', '3.5/4.0', '3',
       'B-plus', '50/100', '7.2/10'], dtype=object)

<div class="alert alert-warning">This looks pretty messy.</div>